EDAF75 - notebook for lab 1
===========================

**Author:** Christian Söderberg



## SQL queries



This lab is meant to be run as a `jupyter` notebook, you can download it from the course website (the `lab1.zip` archive contains the notebook and the database).

To start your notebook, first you have to unzip `lab1.zip`, and then start `jupyter` where it can find your unpacked files.
If you install `jupyter` using `pixi` (which is what I would recommend), then you can use the following incantation (this information is obviously unnecessary if you're already running the notebook, but can be useful if you're reading this as a regular webpage&#x2026;):

<pre><code>
$ unzip lab1.zip
$ cd lab1
$ pixi init
$ pixi add jupyterlab jupysql
$ pixi run jupyter lab
</code></pre>

This should start jupyter in a browser tab, and there you can click
`lab1.ipynb`.

If you already have installed `jupyter` in some other way, you can just start it inside the `lab1` directory.



### Background



We have a database to handle the academic achievements of students at LTH &#x2013; in it we have four tables:

<center>
<img src='./lab1-er.svg'>
</center>

-   `students`
    -   `ssn`: social security number (personnummer)
    -   `first_name`: first name (duh!)
    -   `last_name`: last name

-   `departments`
    -   `department_code`: unique code for each department, such as `eda`, `fma`, &#x2026;
    -   `department_name`: the name of the department, in Swedish

-   `courses`
    -   `course_code`: course code (like `edaf75`)
    -   `course_name`: the name of the course, in Swedish (like "Databasteknik")
    -   `department_code`: the department giving the course
    -   `level`: the course level &#x2013; can be either "G1", "G2", or "A"
    -   `credits`: the number of credits for the course (like 7.5 for `edaf75`)

-   `finished_courses`
    -   `ssn`: the `ssn` of the student who has finished a course
    -   `course_code`: the `course_code` for the finished course
    -   `grade`: the grade for the student finishing the course

Some sample data:

<pre>
<code>ssn            first&#95;name  last&#95;name</code>
<code>-----------    ----------  ---------</code>
<code>19950705-2308  Anna        Johansson</code>
<code>19930702-3582  Anna        Johansson</code>
<code>19911212-1746  Emma        Alm</code>
<code>...          ...         ...</code>
<code></code>
<code>department&#95;code  department&#95;name</code>
<code>---------------  ----------------------------------------</code>
<code>eda              Datavetenskap</code>
<code>edi              Informationsteori</code>
<code>eem              Elektrisk mätteknik</code>
<code>...              ...</code>
<code></code>
<code>course&#95;code  course&#95;name           </code> <code>department&#95;code  level  credits</code>
<code>-----------  ---------------------  ---------------  ----- </code> --&#x2013;&#x2014;
<code>EDA016       Programmeringsteknik   eda              G1     7.5</code>
<code>EDA031       C++ - programmering    eda              G2     7.5</code>
<code>EDA040       Realtidsprogrammering  eda              G2     6.0</code>
<code>...          ...                    ...              ...    ...</code>
<code></code>
<code>ssn            course&#95;code  grade</code>
<code>-----------    -----------  -----</code>
<code>19950705-2308  EITN35       5</code>
<code>19950705-2308  ESS050       3</code>
<code>19950705-2308  ETIN70       4</code>
<code>...          ...          ...</code>
</code></pre>

The tables have been created with the following SQL statements:

<pre>
<code>CREATE TABLE students (</code>
<code>  ssn          CHAR(11),</code>
<code>  first&#95;name   TEXT NOT NULL,</code>
<code>  last&#95;name    TEXT NOT NULL,</code>
<code>  PRIMARY KEY  (ssn)</code>
<code>);</code>
<code></code>
<code>CREATE TABLE departments (</code>
<code>  department&#95;code  TEXT,</code>
<code>  department&#95;name  TEXT,</code>
<code>  PRIMARY KEY      (department&#95;code)</code>
<code>);</code>
<code></code>
<code>CREATE TABLE courses (</code>
<code>  course&#95;code      CHAR(6),</code>
<code>  course&#95;name      TEXT NOT NULL,</code>
<code>  department&#95;code  TEXT,</code>
<code>  level            CHAR(2),</code>
<code>  credits          DOUBLE NOT NULL CHECK (credits > 0),</code>
<code>  PRIMARY KEY      (course&#95;code),</code>
<code>  FOREIGN KEY      (department&#95;code) REFERENCES</code> <code>departments(department&#95;code)</code>
<code>);</code>
<code></code>
<code>CREATE TABLE finished&#95;courses (</code>
<code>  ssn           CHAR(11),</code>
<code>  course&#95;code   CHAR(6),</code>
<code>  grade         INTEGER NOT NULL CHECK (grade >= 3 AND grade</code> <= 5),
<code>  PRIMARY KEY   (ssn, course&#95;code),</code>
<code>  FOREIGN KEY   (ssn) REFERENCES students(ssn),</code>
<code>  FOREIGN KEY   (course&#95;code) REFERENCES</code> <code>courses(course&#95;code)</code>
<code>);</code>
</pre>

All courses offered at the "Computer Science and Engineering" program at LTH during the academic year 2013/14 are in the table `courses`.
Also, the database has been filled with made up data.
SQL statements like the following have been used to insert the data:

<pre>
<code>INTO   students(ssn, first&#95;name, last&#95;name)</code>
<code>VALUES ('19950705-2308', 'Anna', 'Johansson'),</code>
<code>       ('19930702-3582', 'Anna', 'Johansson'),</code>
<code>       ('19911212-1746', 'Emma', 'Alm'),</code>
<code>       ('19910707-3787', 'Emma', 'Nilsson'),</code>
<code>       ...</code>
</pre>



### Assignments



As said above, this lab is designed to be run as a jupyter notebook.
If you haven't got jupyter running, you can run the sql-commands for the problems below interactively inside `sqlite3`, or as a script file with all your commands, or inside some IDE, like [DB Browser for SQLite](https://sqlitebrowser.org/).

If you're running the lab as a notebook, evaluate the following cells before you begin:



In [62]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


If we install using `pixi` as show above, we're going to use `jupysql` (instead of `ipython-sql`), and we'll get a default limit of 10 rows for any query &#x2013; to turn this limit off we can write:



In [63]:
%config SqlMagic.displaylimit = None

We can now load the lab database:



In [64]:
%sql sqlite:///lab1.sqlite

The tables `students`, `departments`, `courses` and `finished_courses` are already in your database, you can see some of their contents by running the cells below:



In [65]:
%%sql
SELECT  *
FROM    students
LIMIT   4

 * sqlite:///lab1.sqlite
Done.


ssn,first_name,last_name
19950705-2308,Anna,Johansson
19930702-3582,Anna,Johansson
19911212-1746,Emma,Alm
19910707-3787,Emma,Nilsson


In [66]:
%%sql
SELECT  *
FROM    departments
LIMIT   4

 * sqlite:///lab1.sqlite
Done.


department_code,department_name
eda,Datavetenskap
edi,Informationsteori
eem,Elektrisk mätteknik
eie,Industriell elektroteknik och automation


In [67]:
%%sql
SELECT  *
FROM    courses
LIMIT   4

 * sqlite:///lab1.sqlite
Done.


course_code,course_name,department_code,level,credits
EDA016,Programmeringsteknik,eda,G1,7.5
EDA031,C++ - programmering,eda,G2,7.5
EDA040,Realtidsprogrammering,eda,G2,6.0
EDA050,Operativsystem,eda,G2,4.5


In [68]:
%%sql
SELECT  *
FROM    finished_courses
LIMIT   4

 * sqlite:///lab1.sqlite
Done.


ssn,course_code,grade
19950705-2308,EITN35,5
19950705-2308,ESS050,3
19950705-2308,ETIN70,4
19950705-2308,FMA140,4


If you inadvertently change the contents of the tables, you can always
recreate the them with the following command (it must be run at the
command line):

    $ sqlite3 lab1.sqlite < create-lab1-db.sql



#### Warm up problems



The problems in this section will not be reviewed during the lab session, they're just to get you up to speed.



##### Problem 1 (practice)



What are the names (first name and last name) of all the students?



In [69]:
%%sql
SELECT first_name, last_name
FROM students

 * sqlite:///lab1.sqlite
Done.


first_name,last_name
Anna,Johansson
Anna,Johansson
Emma,Alm
Emma,Nilsson
Elaine,Robertson
Maria,Nordman
Helena,Troberg
Lotta,Emanuelsson
Anna,Nyström
Maria,Andersson


Now sort the names, first by last name and then by first name:



In [70]:
%%sql
SELECT first_name, last_name
FROM students
ORDER BY last_name, first_name

 * sqlite:///lab1.sqlite
Done.


first_name,last_name
Daniel,Ahlman
Emma,Alm
Martin,Alm
Erik,Andersson
Erik,Andersson
Maria,Andersson
Niklas,Andersson
Märit,Aspegren
Daniel,Axelsson
Henrik,Berg


When you get it to work, experiment by listing only the 10 first students (alphabetically), then try to list only students 11-20, etc.:



In [71]:
%%sql
SELECT first_name, last_name
FROM students
ORDER BY last_name, first_name
LIMIT 10 OFFSET 10

 * sqlite:///lab1.sqlite
Done.


first_name,last_name
Lisa,Berg
Johan,Brattberg
Roger,Brorsson
David,Carlsson
Javier,Chan
Bo,Ek
Bo,Ek
Bo,Ek
Karolin,Ek
Lotta,Emanuelsson


##### Problem 2 (practice)



What are the names of the students who were born in 1985?



In [72]:
%%sql
SELECT first_name, last_name
FROM students
WHERE SUBSTRING(ssn, 1, 4) = '1985'

 * sqlite:///lab1.sqlite
Done.


first_name,last_name
Ulrika,Jonsson
Bo,Ek
Filip,Persson
Henrik,Berg


Hint: the `substring` function can be useful (it also goes by the name
`substr`).



##### Problem 3 (practice)



The penultimate digit in the social security number is even for females, and odd for males. List the first names of all female students in our database alphabetically.



In [73]:
%%sql
SELECT first_name
FROM students
WHERE SUBSTRING(ssn, 12, 1) % 2 = 0

 * sqlite:///lab1.sqlite
Done.


first_name
Anna
Anna
Emma
Emma
Elaine
Maria
Helena
Lotta
Anna
Maria


Now try to output each name only once (so, no duplicates).



In [74]:
%%sql
SELECT DISTINCT first_name
FROM students
WHERE SUBSTRING(ssn, 12, 1) % 2 = 0

 * sqlite:///lab1.sqlite
Done.


first_name
Anna
Emma
Elaine
Maria
Helena
Lotta
Marie
Tina
Salome
Märit


Now try to output the number of distinct names in the listing of female students above, using `count`. It turns out that this is a bit tricky, we need to make sure we use the word `DISTINCT` in the right place (look carefully in the [documentation](https://sqlite.org/lang_aggfunc.html)).



In [75]:
%%sql
SELECT COUNT(DISTINCT first_name)
FROM students
WHERE SUBSTRING(ssn, 12, 1) % 2 = 0

 * sqlite:///lab1.sqlite
Done.


COUNT(DISTINCT first_name)
21


Write a query to show the number of female students, and the number of male students. We want to get the following output:

<center>
<div>
<table rules="all">
<TR>
  <TH>Gender</TH>
  <TH>Count</TH>
</TR>
<TR>
  <TD>female</TD>
  <TD>25</TD>
</TR>
  <TR><TD>male</TD>
  <TD>47</TD>
</TR>
</table>
</div>
</center>

To do this, we can use the `CASE` - `WHEN` construct (see lecture 1) &#x2013; in this case we want to create the two categories `'female'` and `'male'`, and then find a way to count the number of rows in each category.



In [76]:
%%sql
SELECT
    CASE
        WHEN SUBSTRING(ssn, 12, 1) % 2 = 0 THEN 'female'
        ELSE 'male'
    END AS gender,
    COUNT() AS count
FROM students
GROUP BY gender

 * sqlite:///lab1.sqlite
Done.


gender,count
female,26
male,46


##### Problem 4 (practice)



In the next few queries, we'll look at the results of the student with the social security number `19910101-1234` &#x2013; to make things *a lot* easier, start by creating a `VIEW` with all his results.



In [77]:
%%sql
CREATE VIEW results_better AS
    SELECT course_code, grade
    FROM finished_courses
    WHERE ssn = '19910101-1234'

 * sqlite:///lab1.sqlite
(sqlite3.OperationalError) view results_better already exists
[SQL: CREATE VIEW results_better AS
    SELECT course_code, grade
    FROM finished_courses
    WHERE ssn = '19910101-1234']
(Background on this error at: https://sqlalche.me/e/20/e3q8)


Make sure the view contains all data pertinent to the student in question (it will make the following queries very simple).

Which courses (course codes only) have been taken by the student?



In [78]:
%%sql
SELECT course_code, grade
FROM results_better

 * sqlite:///lab1.sqlite
Done.


course_code,grade
EDA070,3
EDA385,5
EDAA25,4
EDAF05,3
EEMN10,5
EIT020,3
EIT060,4
EITF40,5
EITN40,3
EITN50,4


What are the names of these courses, and how many credits do they give?



In [79]:
%%sql
SELECT course_name, credits
FROM courses
WHERE course_code IN
    (SELECT course_code
     FROM results_better)

 * sqlite:///lab1.sqlite
Done.


course_name,credits
Datorer och datoranvändning,3.0
"Konstruktion av inbyggda system, fördjupningskurs",7.5
C-programmering,3.0
"Algoritmer, datastrukturer och komplexitet",5.0
Datorbaserade mätsystem,7.5
Digitalteknik,9.0
Datasäkerhet,7.5
Digitala och analoga projekt,7.5
Avancerad webbsäkerhet,4.0
Avancerad datasäkerhet,7.5


How many credits has the student taken?



In [80]:
%%sql
SELECT SUM(credits)
FROM courses
WHERE course_code IN
    (SELECT course_code
     FROM results_better)

 * sqlite:///lab1.sqlite
Done.


SUM(credits)
249.5


What is the student's grade average? It turns out that there are actually (at least) two different averages at play here:

-   the unweighted average, i.e., just the average of all the students
    grades (no matter how many credits each course gives), and

-   the [weighted average](https://en.wikipedia.org/wiki/Weighted_arithmetic_mean), where we use the credits for a course as a weight.

First the unweighted average:



In [81]:
%%sql
SELECT AVG(grade) AS GPA
FROM results_better

 * sqlite:///lab1.sqlite
Done.


GPA
4.0285714285714285


And then the weighted average (feel free to ask me about this during QA
sessions, if you get stuck):



In [92]:
%%sql
CREATE VIEW results_best AS
    SELECT course_code, grade,
        (SELECT credits
         FROM courses c
         WHERE c.course_code = finished_courses.course_code) AS credits
    FROM finished_courses
    WHERE ssn = '19910101-1234'

 * sqlite:///lab1.sqlite
Done.


[]

In [91]:
%%sql
DROP VIEW results_best

 * sqlite:///lab1.sqlite
Done.


[]

Hint: If you've created a proper view above, we'll get a 'table' with
one row for each course the student has finished, and each row will
contain information about grades and credits for the finished course.
If we use arithmetic operations in a select statement, and then use an
aggregate function around that operation, we'll apply the aggregate
function to each value the operation returns (so, e.g., a `sum` over a
product will be a scalar product).

Now drop the view:



In [93]:
%%sql
SELECT 
    (SUM(grade*credits)) AS "all weighted", 
    (SUM(credits)) AS "total credits", 
    (SUM(grade*credits)) / (SUM(credits)) AS GPA
FROM results_best


 * sqlite:///lab1.sqlite
Done.


all weighted,total credits,GPA
1032.5,249.5,4.138276553106213


---



#### Review problems



##### Problem 5 - REVIEW



List the 10 first students in birthday order (not by age, just the birthday &#x2013; the birthday is a part of the social security number).

The output should begin with:

<center>
<div>
<table rules="all">
<TR>
  <TH>Ssn</TH>
  <TH>First name</TH>
  <TH>Last name</TH>
</TR>
<TR>
  <TD>19910101-1234</TD>
  <TD>Peter</TD>
  <TD>Solberg</TD>
</TR>
<TR>
  <TD>19890103-1256</TD>
  <TD>Johan</TD>
  <TD>Brattberg</TD>
</TR>
<TR>
  <TD>19950125-1153</TD>
  <TD>Magnus</TD>
  <TD>Hultgren</TD>
</TR>
<TR>
  <TD>19900129-3374</TD>
  <TD>Henrik</TD>
  <TD>Gustavsson</TD>
</TR>
<TR>
  <TD>19860206-1065</TD>
  <TD>Eva</TD>
  <TD>Hjort</TD>
</TR>
</table>
</div>
</center>

Hint: You can use the `substring` function to pick out the birthday from the `ssn`.



In [85]:
%%sql
SELECT ssn, first_name, last_name
FROM students
ORDER BY substring(ssn, 5, 4)
LIMIT 10

 * sqlite:///lab1.sqlite
Done.


ssn,first_name,last_name
19910101-1234,Peter,Solberg
19890103-1256,Johan,Brattberg
19950125-1153,Magnus,Hultgren
19900129-3374,Henrik,Gustavsson
19860206-1065,Emma,Hjort
19900206-3735,Daniel,Ahlman
19880206-1915,Henrik,Hult
19850208-1213,Henrik,Berg
19910226-3409,Karin,Östberg
19900227-2369,Märit,Aspegren


Then list all shared birthdays, order first by how many students share the birthday, then by the birthday itself (alphabetically):



In [96]:
%%sql
SELECT substring(ssn, 5, 4) AS Birthday, COUNT(*) AS "Nr of people"
FROM students
GROUP BY Birthday HAVING COUNT(*) > 1
ORDER BY COUNT(*) DESC, Birthday

 * sqlite:///lab1.sqlite
Done.


Birthday,Nr of people
0206,3
0308,2
0819,2
0915,2
1015,2
1030,2


The output should be:

<center>
<div>
<table rules="all">
<TR>
  <TH>Birthday</TH>
  <TH>Nbr of people</TH>
</TR>
<TR>
  <TD>0206</TD>
  <TD>3</TD>
</TR>
<TR><TD>0308</TD>
<TD>2</TD>
</TR>
<TR><TD>0819</TD>
<TD>2</TD>
</TR>
<TR><TD>0915</TD>
<TD>2</TD>
</TR>
<TR><TD>1015</TD>
<TD>2</TD>
</TR>
<TR><TD>1030</TD>
<TD>2</TD>
</TR>
</table>
</div>
</center>



##### Problem 6 &#x2013; REVIEW



How many courses are there for each level (`G1`, `G2`, and `A`)?



In [87]:
%%sql
SELECT COUNT(*) AS "Nr of courses", level
FROM courses
GROUP BY level
ORDER BY COUNT(*)

 * sqlite:///lab1.sqlite
Done.


Nr of courses,level
31,G1
60,G2
87,A


In [ ]:
%%sql
SELECT c.course_code, c.department_code, d.department_name, d.department_code
FROM courses c
JOIN departments d ON d.department_code = c.department_code

For all levels, list all departments who have more than 5 courses at that level.



In [109]:
%%sql
SELECT c.level, d.department_name, COUNT(*) AS "Nr of courses"
FROM courses c
JOIN departments d ON d.department_code = c.department_code
GROUP BY c.level, d.department_code HAVING COUNT(*) > 5
ORDER BY c.level, "Nr of courses" DESC

 * sqlite:///lab1.sqlite
Done.


level,department_name,Nr of courses
A,Elektro- och informationsteknik,26
A,Matematik,16
A,Datavetenskap,14
A,Matematisk statistik,7
A,Reglerteknik,6
G1,Språk och litteraturcentrum,12
G1,Elektro- och informationsteknik,6
G2,Elektro- och informationsteknik,15
G2,Datavetenskap,13


The output should be something like this (but it's totally fine if you list A-courses before the G1/G2 courses).

<center>
<div>
<table rules="all">
<TR><TH>Nivå</TH>
<TH>Institution</TH>
<TH>Antal</TH>
</TR>
<TR><TD>G1</TD>
<TD>Språk och litteraturcentrum</TD>
<TD>12</TD>
</TR>
<TR><TD>G1</TD>
<TD>Elektro- och informationsteknik</TD>
<TD>6</TD>
</TR>
<TR><TD>G2</TD>
<TD>Elektro- och informationsteknik</TD>
<TD>15</TD>
</TR>
<TR><TD>G2</TD>
<TD>Datavetenskap</TD>
<TD>13</TD>
</TR>
<TR><TD>A</TD>
<TD>Elektro- och informationsteknik</TD>
<TD>26</TD>
</TR>
<TR><TD>A</TD>
<TD>Matematik</TD>
<TD>16</TD>
</TR>
<TR><TD>A</TD>
<TD>Datavetenskap</TD>
<TD>14</TD>
</TR>
<TR><TD>A</TD>
<TD>Matematisk statistik</TD>
<TD>7</TD>
</TR>
<TR><TD>A</TD>
<TD>Reglerteknik</TD>
<TD>6</TD>
</TR>
</table>
</div>
</center>

*Optional*: If you really want the A-level courses listed after the G1/G2 in the output (but you're not required to!), you can use a `WITH` statement defining a table with two columns, a level and the corresponding 'order' (so, G1 has order 1, G2 has order 2 and A has order 3), and then join in this ordering and use it for ordering the rows in the output.
To do this you can use a `WITH` with a `VALUES` body (see the SQLite documentation for the [WITH clause](https://sqlite.org/lang_with.html)).



##### Problem 7 - REVIEW



For the ten departments which offer the most total credits (for its
courses in this database) &#x2013; output the name of the department, the
total offered credits, and the number of courses, order by credits, with highest total first:



In [116]:
%%sql
SELECT d.department_name, SUM(c.credits) AS "Totalpoäng", COUNT(*) AS "Antal kurser"
FROM courses c
JOIN departments d ON d.department_code = c.department_code
GROUP BY d.department_code
ORDER BY "Totalpoäng" DESC, "Antal kurser" DESC
LIMIT 10

 * sqlite:///lab1.sqlite
Done.


department_name,Totalpoäng,Antal kurser
Elektro- och informationsteknik,332.0,47
Datavetenskap,216.0,32
Matematik,160.0,26
Språk och litteraturcentrum,106.5,13
Matematisk statistik,84.0,11
Designvetenskaper,73.5,10
Reglerteknik,65.5,9
Elektrisk mätteknik,45.0,6
Fysik,44.5,6
Industriell elektroteknik och automation,27.5,3


The output should begin something like:

<center>
<div>
<table rules="all">
<TR><TH>Institution</TH>
<TH>Totalpoäng</TH>
<TH>Antal kurser</TH>
</TR>
<TR><TD>Elektro- och informationsteknik</TD>
<TD>332.0</TD>
<TD>47</TD>
</TR>
<TR><TD>Datavetenskap</TD>
<TD>216.0</TD>
<TD>32</TD>
</TR>
<TR><TD>Matematik</TD>
<TD>160.0</TD>
<TD>26</TD>
</TR>
<TR><TD>Språk och litteraturcentrum</TD>
<TD>106.5</TD>
<TD>13</TD>
</TR>
<TR><TD>Matematisk statistik</TD>
<TD>84.0</TD>
<TD>11</TD>
</TR>
<TR><TD>...</TD>
<TD>...</TD>
<TD>...</TD>
</TR>
</table>
</div>
</center>



##### Problem 8 - REVIEW



Which students (`ssn` and full name) have taken 0 credits &#x2013; list the students in alphabetical order (last name and then first name)?
This problem can be solved in several ways, first do it with an outer join:



In [123]:
%%sql
SELECT s.ssn, s.first_name, s.last_name
FROM students s
LEFT JOIN finished_courses f ON f.ssn = s.ssn
WHERE f.ssn IS NULL
ORDER BY s.last_name, s.first_name

 * sqlite:///lab1.sqlite
Done.


ssn,first_name,last_name
19891220-1393,Erik,Andersson
19900313-2257,Erik,Andersson
19891007-3091,Roger,Brorsson
19931225-3158,Bo,Ek
19900129-3374,Henrik,Gustavsson
19880206-1915,Henrik,Hult
19950125-1153,Magnus,Hultgren
19911015-3758,Jonathan,Jönsson
19870909-3367,Tina,Lööf
19950829-1848,Anna,Nyström


Do the same thing using a subquery:



In [125]:
%%sql
SELECT s.ssn, s.first_name, s.last_name
FROM students s
WHERE NOT EXISTS (
    SELECT 1
    FROM finished_courses f
    WHERE f.ssn = s.ssn
)
ORDER BY s.last_name, s.first_name

 * sqlite:///lab1.sqlite
Done.


ssn,first_name,last_name
19891220-1393,Erik,Andersson
19900313-2257,Erik,Andersson
19891007-3091,Roger,Brorsson
19931225-3158,Bo,Ek
19900129-3374,Henrik,Gustavsson
19880206-1915,Henrik,Hult
19950125-1153,Magnus,Hultgren
19911015-3758,Jonathan,Jönsson
19870909-3367,Tina,Lööf
19950829-1848,Anna,Nyström


Now just copy/paste one of your solutions above, and use a *window function* to add a column which shows the 'age order' of all the students in the list, the oldest student should get number 1 (but still be listed in alphabetical order).



In [133]:
%%sql
SELECT 
    s.ssn, 
    s.first_name, 
    s.last_name, 
    RANK() OVER (ORDER BY s.ssn) AS "Age rank"
FROM students s
WHERE NOT EXISTS (
    SELECT 1
    FROM finished_courses f
    WHERE f.ssn = s.ssn
)
ORDER BY s.last_name, s.first_name

 * sqlite:///lab1.sqlite
Done.


ssn,first_name,last_name,Age rank
19891220-1393,Erik,Andersson,5
19900313-2257,Erik,Andersson,7
19891007-3091,Roger,Brorsson,4
19931225-3158,Bo,Ek,9
19900129-3374,Henrik,Gustavsson,6
19880206-1915,Henrik,Hult,3
19950125-1153,Magnus,Hultgren,10
19911015-3758,Jonathan,Jönsson,8
19870909-3367,Tina,Lööf,2
19950829-1848,Anna,Nyström,11


The output should be something like:

<center>
<div>
<table rules="all">
<TR><TH>Personnummer</TH>
<TH>Namn</TH>
<TH>Åldersordning</TH>
</TR>
<TR><TD>19891220-1393</TD>
<TD>Erik Andersson</TD>
<TD>5</TD>
</TR>
<TR><TD>19900313-2257</TD>
<TD>Erik Andersson</TD>
<TD>7</TD>
</TR>
<TR><TD>19891007-3091</TD>
<TD>Roger Brorsson</TD>
<TD>4</TD>
</TR>
<TR>
<TD>...</TD>
<TD>...</TD>
</TR>
<TR><TD>19850517-2597</TD>
<TD>Filip Persson</TD>
<TD>1</TD>
</TR>
</table>
</div>
</center>



##### Problem 9 - REVIEW



List the names and average grades of the 10 students with the highest
grade average?
You can use the unweighted average.



In [148]:
%%sql
SELECT 
    f.ssn, 
    s.first_name, 
    s.last_name, 
    ROUND(AVG(grade), 2) AS "GPA"
FROM finished_courses f
JOIN students s ON s.ssn = f.ssn
GROUP BY f.ssn
ORDER BY "GPA" DESC
LIMIT 10

 * sqlite:///lab1.sqlite
Done.


ssn,first_name,last_name,GPA
19861103-2438,Bo,Ek,4.35
19910308-1826,Helena,Troberg,4.31
19931213-2824,Elaine,Robertson,4.24
19930702-3582,Anna,Johansson,4.23
19931208-3605,Ylva,Jacobsson,4.22
19950705-2308,Anna,Johansson,4.2
19920812-1857,Jakob,Malmberg,4.17
19940801-2971,Mikael,Nilsson,4.17
19860819-2864,Maria,Andersson,4.16
19890621-3057,Ola,Nilsson,4.15


The output should begin with:

<center>
<div>
<table rules="all">
<TR><TH>Personnummer</TH>
<TH>Förnamn</TH>
<TH>Efternamn</TH>
<TH>Snittbetyg</TH>
</TR>
<TR><TD>19861103-2438</TD>
<TD>Bo</TD>
<TD>Ek</TD>
<TD>4.35</TD>
</TR>
<TR><TD>19910308-1826</TD>
<TD>Helena</TD>
<TD>Troberg</TD>
<TD>4.31</TD>
</TR>
<TR><TD>19931213-2824</TD>
<TD>Elaine</TD>
<TD>Robertson</TD>
<TD>4.24</TD>
</TR>
<TR><TD>19930702-3582</TD>
<TD>Anna</TD>
<TD>Johansson</TD>
<TD>4.23</TD>
</TR>
<TR><TD>19931208-3605</TD>
<TD>Ylva</TD>
<TD>Jacobsson</TD>
<TD>4.22</TD>
</TR>
<TR><TD>...</TD>
<TD>...</TD>
<TD>...</TD>
<TD>...</TD>
</TR>
</table>
</div>
</center>

It's OK to use more digits in the output, but using the `format`-function it's pretty easy to get the numbers formatted as above.



##### Problem 10 - REVIEW



List the social security number and total number of credits for all
students &#x2013; order by total credits, descending.
Students with no credits
should be included in the listing, with 0 credits (not `NULL`).

Use an outer `JOIN` to solve the problem &#x2013; you might want to use the
function `coalesce(v1, v2, ...)`; it returns the first value which is
not `NULL`, so `coalesce(avg(grade), 0)` would give 0 if the were no
grades (i.e., if `grade` were `NULL`), you can also try the `ifnull`
function.



In [163]:
%%sql
SELECT 
    s.ssn, 
    COALESCE(SUM(c.credits), 0) AS "total"
FROM students s
LEFT JOIN finished_courses f ON s.ssn = f.ssn
LEFT JOIN courses c ON c.course_code = f.course_code
GROUP BY s.ssn
ORDER BY "total" DESC

 * sqlite:///lab1.sqlite
Done.


ssn,total
19951004-2346,350.0
19880620-2564,348.5
19910915-2068,338.0
19920623-3258,334.0
19921222-2113,332.0
19890621-3057,295.5
19920308-3854,289.0
19930702-3582,288.5
19921029-1995,268.5
19920921-2499,267.5


##### Problem 11 - REVIEW



Do all students have unique names (first name *and* last name)? If not,
show the full name and social security number for all students who have
a namesake.

As usual there are several ways of solving this, solve it using a
`WITH`-statement where you create a 'table' with all duplicate names,
and then:

Use a `JOIN`:



In [165]:
%%sql
WITH duplicates AS (
    SELECT first_name, last_name
    FROM students
    GROUP BY first_name, last_name
    HAVING COUNT(*) > 1
)

SELECT s.ssn, s.first_name, s.last_name
FROM students s
JOIN duplicates d
    ON s.first_name = d.first_name
    AND s.last_name = d.last_name
ORDER BY s.last_name, s.first_name, s.ssn

 * sqlite:///lab1.sqlite
Done.


ssn,first_name,last_name
19891220-1393,Erik,Andersson
19900313-2257,Erik,Andersson
19850819-2139,Bo,Ek
19861103-2438,Bo,Ek
19931225-3158,Bo,Ek
19930702-3582,Anna,Johansson
19950705-2308,Anna,Johansson


Use a subquery:



In [169]:
%%sql
SELECT s1.ssn, s1.first_name, s1.last_name
FROM students s1
WHERE EXISTS(
    SELECT 1
    FROM students s2
    WHERE s2.first_name = s1.first_name
        AND s2.last_name = s1.last_name
        AND s2.ssn != s1.ssn
)
ORDER BY s1.last_name, s1.first_name, s1.ssn

 * sqlite:///lab1.sqlite
Done.


ssn,first_name,last_name
19891220-1393,Erik,Andersson
19900313-2257,Erik,Andersson
19850819-2139,Bo,Ek
19861103-2438,Bo,Ek
19931225-3158,Bo,Ek
19930702-3582,Anna,Johansson
19950705-2308,Anna,Johansson
